In [10]:
import pandas as pd
import csv
import json
import os
from dotenv import load_dotenv
from collections import defaultdict
from openai import OpenAI

In [11]:
caminho_csv = "Data\\dados_processados_projetos_final.csv"
df = pd.read_csv(caminho_csv)

In [12]:
load_dotenv(override=True)

api_key = os.getenv('OPENAI_API_KEY')

In [13]:
openai = OpenAI()

In [14]:
'''
    --> Prompt
'''

def gerar_prompt(dado_dict, quantidade):

    dados_gerais_dict = {
    "Functional requirement (US Description)": dado_dict['US'][0]["Descritivo_da_US"],
    "Domain": dado_dict['US'][0]["Dominio"],
    "Platform": dado_dict['US'][0]["Plataforma"],
    "Architecture": dado_dict['US'][0]["Arquitetura"]
    }
    tarefas = [{
        "Tarefa_ID": dado_dict['US'][i]['Tarefa_ID'],
        "Modulo": dado_dict['US'][i]['Modulo'],
        "Operacao": dado_dict['US'][i]['Operacao'],
        "Tarefa_mapeada": dado_dict['US'][i]['Tarefa_mapeada'],
        "Tarefa_original": dado_dict['US'][i]['Tarefa_original'],
        "Tags": dado_dict['US'][i]['Tags'],
        "Camada": dado_dict['US'][i]['Camada'],
        "Linguagem": dado_dict['US'][i]['Linguagem'],
        "Framework": dado_dict['US'][i]['Framework'],
        "Outras_Tags": dado_dict['US'][i]['Outras_Tags'],
    }for i in range(quantidade)]
    
    base_prompt = f"""
Given the structured project data dictionary called `dado`, analyze and recommend one or more non-functional requirements (NFRs) that fit the scenario. Only recommend NFRs belonging to the following types and attributes:

- Performance: response_time, capacity, transit_delay, efficiency_compliance  
- Reliability: availability, integrity, fault_tolerance, recoverability  
- Security: confidentiality, access_control, authentication  

Follow this step-by-step reasoning:

1. Read the project data from the `dado` dictionary:
   - US General Data: {dados_gerais_dict}
   - Associated tasks: {tarefas}

2. Understand the functional requirement (US Description) and its sensitivity or business implications.

3. Analyze each field of the `dado` dictionary to understand its impact on NFR selection:

   - `Functional requirement (US Description)`: Provides the core user story — for example, creating user permissions — which may highlight the need for security-related NFRs such as access control and authentication.
   - `Domain`: Specifies the context in which the system operates — e.g., "Office/Business" — which can influence the importance of confidentiality and system responsiveness.
   - `Platform`: Indicates the deployment environment — such as "Web" — which suggests a need for high availability and acceptable response times under concurrent access.
   - `Architecture`: Describes the system structure — e.g., "Client-server" — which informs the selection of NFRs like fault tolerance, recoverability, and efficiency across distributed components.
   - `Associated_tasks`: A list of technical implementation tasks that provide insight into:
     - The layers involved (e.g., Front-end in Angular/TypeScript, Back-end in Java/Spring Boot),
     - The operations performed (e.g., validating user permissions, creating listing services),
     - The technical stack, which influences how NFRs like performance or security should be enforced.
     This field is key to understanding where to apply specific NFRs (e.g., enforce access control in front-end validation, ensure back-end service availability).

4. Based on the full context, select the most appropriate NFR type(s) and attribute(s) from the allowed list.

5. For each, draft one clear and relevant NFR sentence aligned with the chosen attribute.  
   - Format: "The system must ensure the [ATTRIBUTE] by [MEASURE OR CONDITION]."

6. Format the final output as a JSON array using the structure below. Generate **one item per NFR recommendation:

[
  {{
    "NFR_Tipo": "[Performance|Reliability|Security]",
    "NFR_Atributo": "[attribute from allowed list]",
    "NFR_Sentença": "The system must ensure the [ATTRIBUTE] by [MEASURE OR CONDITION]."
  }},
  ...
]

Important:

- You may recommend more than one NFR if justified by the input.
- Return only a valid JSON array — no explanations, no comments, no extra text.
- The recommendation must contain more non-functional requirements than the number of tasks present in the Sprint and US.
"""
    return base_prompt

In [15]:
def openai_api(prompt):

    response = openai.chat.completions.create(
            model="gpt-4.1-mini",
            messages=[
                {"role": "system", "content": "You are a requirements engineer. Use chain-of-thought reasoning to analyze the user's input."},
                {"role": "user", "content": prompt}
            ]
        )
    resposta_str = response.choices[0].message.content.strip()
    return resposta_str

In [16]:
'''
Agrupando os dados por user stories
'''

caminho_csv = "Data\\DadosProjetos.csv"
grupos = defaultdict(list)

with open(caminho_csv, newline='', encoding='utf-8') as csvfile:
    leitor = csv.DictReader(csvfile)
    for linha in leitor:
        chave = (linha["Projeto"], linha["Sprint_ID"], linha["US_ID"])
        grupos[chave].append(linha)
print(len(grupos.items()))

246


'\nfor (projeto, sprint, us), tarefas in grupos.items():\n    print()\n    print(f"O projeto é {projeto}, pertence a sprint {sprint} e é da US é {us}")\n    print("----------------------------------")\n    print(tarefas)\n    print("\n----------------------------------\n")\n'

In [17]:
resultados = []

def gerar_recomendacoes_funcao_2():
    try:
        caminho_saida = "result/response.json"
        
        for (projeto, sprint, us), tarefas in grupos.items():
            payload = {
                "Projeto": projeto,
                "Sprint_ID": sprint,
                "US_ID": us,
                "US": tarefas
            }
    
            quantidade = len(payload['US'])
            prompt = gerar_prompt(payload, quantidade)
    
            resposta = openai_api(prompt)
            
            try:
                resposta_json = json.loads(resposta)
            except json.JSONDecodeError as e:
                print(f"Erro ao decodificar a resposta como JSON: {e}")
                resposta_json = [{"erro": "Resposta inválida", "resposta_bruta": resposta}]

            resultados.append({
                "Projeto": projeto,
                "Sprint_ID": sprint,
                "US_ID": us,
                "Resposta": resposta
            })
            
            with open(caminho_saida, "w", encoding="utf-8") as f:
                json.dump(resultados, f, indent=2, ensure_ascii=False)
    
    except Exception as e:
        print(f"Erro ocorrido: {e}")
        print("Salvando resultados parciais...")
    
    finally:
        print(f"Resultados salvos com {len(resultados)} entradas.")

In [ ]:
gerar_recomendacoes_funcao_2()